### Post processing scripts to generate figures for Fleischer datatset

In [ ]:
import pandas as pd
import numpy as np
import yaml
import os
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Plot settings
sns.set_style(style='white')

plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
plt.rc('text.latex', preamble=r"\usepackage{amsmath}"
           r"\usepackage{amstext}")
plt.rcParams["axes.linewidth"] = 2.50
plt.rcParams['xtick.major.size'] = 20
plt.rcParams['ytick.major.size'] = 20
fsz = 28

In [ ]:
## load the config file
with open('params_cel.yaml', 'rb') as f:
    conf = yaml.safe_load(f.read())   
    
output_folder = conf['datasets']['Fleischer']['output_folder']

### Code to plot chromatin transition energy barrier

In [ ]:
# Let's now plot chromatin transition energy barrier shown in Fig. 9c in the main paper. For supplementary figures shown in Fig S10c, follow the comments below.
# For this we need to read the files generated by running the code above that generates files in the respective output_folder and then append them as a single file to use it for the plotting
fol_name = [x[0] for x in os.walk(output_folder)]
counter = 0
filter = [1,3,5] # We will show the plot for gamma = 3 in the main and for [1,5] in the supplementary
list_mean = []
list_std = []
Chr_label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,"X"]
counter = 0
for fol in fol_name[1:]:
    f = os.path.join(fol, "AUC_energy_df_"+str(fol.split('/')[-1])+".csv")
    chromosome = str(fol.split('/')[-1]).split("_")[1]
    if chromosome == 'X':
        chromosome = np.array(23, dtype=int)
    elif chromosome == 'Y':
        chromosome = np.array(24, dtype=int)
    else:
        chromosome = np.array(chromosome, dtype=int)
    df_energy = pd.read_csv(f)
    labels = df_energy.columns[1:].tolist()
    # We will sum the energy over all chromosomes
    if counter == 0:
        df_select = pd.DataFrame(columns= ['filter_size'] + labels)
        df_select = df_energy.loc[df_energy['filter_size'].isin(filter)].reset_index(drop=True)
    elif chromosome != 24:
        df_select[labels] = df_select[labels] + df_energy.loc[df_energy['filter_size'].isin(filter)][labels].reset_index(drop=True)
    counter = counter + 1


In [ ]:
# Let's now plot transition barrier values for the upper and lower matrix separately
df_sum_ut = []
df_sum_lt = []
df_sum_diff = []
for lab in labels:
    temp = str(lab).split('_')
    if "ut" in temp:
        df_sum_ut.append(list(df_select[lab]*1000))
    elif "lt" in temp:
        df_sum_lt.append(list(df_select[lab]*1000))
    else:
        df_sum_diff.append(list(df_select[lab]*1000))

x1 = [1,2,3,4,5]
fig, ax = plt.subplots()
fig.set_size_inches(10, 8)

df_list_treat = {'Grp_1_20=>Grp_1_20': 'k', 'Grp_21_40=>Grp_1_20': 'k', 'Grp_41_60=>Grp_1_20': 'k', 'Grp_61_80=>Grp_1_20': 'k','Grp_81_100=>Grp_1_20':'k'}
y_ut = []
y_lt = []
y_diff = []
for i in range(0,len(df_sum_ut[0])):
    y_ut.append([line[i] for line in df_sum_ut])
    y_lt.append([line[i] for line in df_sum_lt])
    y_diff.append([line[i] for line in df_sum_diff])

## Let's start plotting
ax.plot(x1,y_diff[1],'--ok',linewidth = 3, markersize=12)
ax.plot(x1,y_ut[1],'-vb',x1,y_lt[1],'-^r',linewidth = 3.5, markersize=12)

# Time to make it more aesthetically pleasing
plt.ylim([-0.8,0.01*1000])
ax.set_ylabel('$\Delta U $', fontsize=fsz+10, color='k')
ax.tick_params('both',which='major', length=7,labelsize=fsz)
ax.tick_params('both',which='major', length=7,labelsize=fsz)
labels1 = []
for lab in labels[0:5]:
    string  = str(lab).split('_')[:-1]
    string = '_'.join(string)
    temp = str(string).split('=>')
    labels1.append(str(temp[0])+ "$\Rightarrow$" +str(temp[1]))
ax.set_xticks(x1)
ax.set_xticklabels(labels1)
ax.grid(False)
# Setting the color for the axis
color = []
for col in labels[0:5]:
    temp = [i for i, s in enumerate(df_list_treat) if s in col]
    color.append(list(df_list_treat.items())[temp[0]][1])
i = 0
for xtick in ax.get_xticklabels():
    xtick.set_color(color[i])
    i = i + 1
plt.xticks(rotation=45,ha='right')
plt.show()
# plt.savefig(os.path.join(output_folder, "mean_chromosome_energy_Supp_param_1.pdf"),format='pdf', bbox_inches='tight',dpi=512)
# plt.close()